## MNIST dataset prediction using Neural Network

In [1]:
import tensorflow as tf

In [2]:
# To prevent the warning from appearinig all the time
import os 
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [3]:
# Check if the GPUS is available for the tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
# Import other libraries that will be used 
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [5]:
# Load mnist dataset 

mnist = keras.datasets.mnist

In [6]:
# split the dataset into training and testing 

(x_train, y_train), (x_test, y_test) = mnist.load_data()

print(f"x_train shape is {x_train.shape}, y_train shape is {y_train.shape}")
print(f"x_test shape is {x_test.shape}, y_test shape is {y_test.shape}")

print(f"Type of x (image)  train is {type(x_train)}")
print(f"Type of y (labels) train is {type(x_train)}")

print(f"Type of x (image)  test is {type(x_test)}")
print(f"Type of y (labels) test is {type(y_test)}")

# We have 60000 images, each image size is 28*28

x_train shape is (60000, 28, 28), y_train shape is (60000,)
x_test shape is (10000, 28, 28), y_test shape is (10000,)
Type of x (image)  train is <class 'numpy.ndarray'>
Type of y (labels) train is <class 'numpy.ndarray'>
Type of x (image)  test is <class 'numpy.ndarray'>
Type of y (labels) test is <class 'numpy.ndarray'>


In [7]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [8]:
import datetime

In [9]:
# Normalize: 0,255 -> 0,1
# Normalize the images to reduce the noise in training
x_train, x_test = x_train / 255.0, x_test / 255.0

In [10]:
# model using keras api
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=(28,28)), # Flatten the input images, it will be convered from 2D to 1D
    keras.layers.Dense(128, activation='relu'), # Dense layer with relu as activation function (Adding non-linearity to the network)
    keras.layers.Dense(10), # Output layer
])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 128)               100480    
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
# another way to build the Sequential model:
#model = keras.models.Sequential()
#model.add(keras.layers.Flatten(input_shape=(28,28))
#model.add(keras.layers.Dense(128, activation='relu'))
#model.add(keras.layers.Dense(10))

In [12]:
# loss and optimizer
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True) # Used for classification problems
optim = keras.optimizers.Adam(learning_rate=0.001) # Deifine Adam as optimizer function
metrics = ["accuracy"] # Add accuracy metric to measure it during training and validation

In [13]:
# Configures the model for training
model.compile(loss=loss, optimizer=optim, metrics=metrics)

In [14]:
# training hyperparameters
batch_size = 64
epochs = 5

In [15]:
# Configure the tensorboard configurations
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [16]:
# Train the model :) 
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, shuffle=True, verbose=1, callbacks=[tensorboard_callback])

Epoch 1/5
938/938 [==============================] - 5s 3ms/step - loss: 0.3007 - accuracy: 0.9148
Epoch 2/5
938/938 [==============================] - 2s 2ms/step - loss: 0.1368 - accuracy: 0.9601
Epoch 3/5
938/938 [==============================] - 2s 2ms/step - loss: 0.0958 - accuracy: 0.9725
Epoch 4/5
938/938 [==============================] - 2s 2ms/step - loss: 0.0721 - accuracy: 0.9789
Epoch 5/5
938/938 [==============================] - 2s 2ms/step - loss: 0.0566 - accuracy: 0.9834


In [17]:
# evaulate the model
model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1, callbacks=[tensorboard_callback])

157/157 [==============================] - 0s 2ms/step - loss: 0.0818 - accuracy: 0.9741


[0.08177682012319565, 0.9740999937057495]

In [18]:
# Load the tensorboard inside the notebook
%tensorboard --logdir logs/fit

In [19]:
# predictions

# 1. option: build new model with Softmax layer
probability_model = keras.models.Sequential([
    model,
    keras.layers.Softmax()
])

predictions = probability_model(x_test)
#import pdb; pdb.set_trace()
# Print sample from the output, tensor for 10 values, probabilities for our classes(numbers from 0 to 10)
pred0 = predictions[0]
print(pred0)

# use np.argmax to get label with highest probability
print(f"The type of pred0 is {type(pred0)}")
label0 = np.argmax(pred0, axis=0) # Returns the indices of the maximum values along an axis.
print(f"The predicted class is {label0}")

tf.Tensor(
[3.08527910e-06 2.50741834e-07 1.14193674e-04 1.94106752e-03
 4.43793802e-09 6.74565513e-07 1.52492671e-10 9.97883260e-01
 5.15430520e-06 5.23323033e-05], shape=(10,), dtype=float32)
The type of pred0 is <class 'tensorflow.python.framework.ops.EagerTensor'>
The predicted class is 7


In [20]:
print("Shape: ", x_test[0].shape)
print("Label: ", y_test[0])


# Using the file writer, log the reshaped image.
logdir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(logdir)

# Reshpae the image to show it inside tensorboard
img = np.reshape(x_test[0], (-1, 28, 28, 1))

with file_writer.as_default():
    tf.summary.image("Test image", img, step=0)

Shape:  (28, 28)
Label:  7


In [21]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 27029), started 0:00:00 ago. (Use '!kill 27029' to kill it.)

In [23]:
# 2. option: original model + nn.softmax, call model(x)
predictions = model(x_test)
predictions = tf.nn.softmax(predictions)
pred0 = predictions[0]
print(f"Probabilites values -> ", pred0)

label0 = np.argmax(pred0)
print(f"The predicted class is {label0}")


Probabilites values ->  tf.Tensor(
[3.08527910e-06 2.50741834e-07 1.14193674e-04 1.94106752e-03
 4.43793802e-09 6.74565513e-07 1.52492671e-10 9.97883260e-01
 5.15430520e-06 5.23323033e-05], shape=(10,), dtype=float32)
The predicted class is 7


In [24]:
# 3. option: original model + nn.softmax, call model.predict(x)
predictions = model.predict(x_test, batch_size=batch_size)
predictions = tf.nn.softmax(predictions)
pred0 = predictions[0]
print(f"Probabilites values -> ", pred0)
label0 = np.argmax(pred0)
print(f"The predicted class is {label0}")

# call argmax for multiple labels
pred05s = predictions[0:5]
print(f"The shape for the output is {pred05s.shape}")
label05s = np.argmax(pred05s, axis=1) # axis 1 means for rows
print(f"The predicted classes are {label05s}")

Probabilites values ->  tf.Tensor(
[3.08527910e-06 2.50741834e-07 1.14193674e-04 1.94106752e-03
 4.43793802e-09 6.74565513e-07 1.52492671e-10 9.97883260e-01
 5.15430520e-06 5.23323033e-05], shape=(10,), dtype=float32)
The predicted class is 7
The shape for the output is (5, 10)
The predicted classes are [7 2 1 0 4]
